# 超参数与模型验证

在上一节中，我们介绍了有监督机器学习模型的基本步骤：
1. 选择模型类；
2. 选择模型超参数；
3. 用模型拟合训练数据；
4. 用模型预测新数据的标签。

前两步——模型选择和超参数选择——可能是有效使用各种机器学习工具和技术的最重要阶段。为了作出正确的选择，我们需要一种方式来验证选中的模型和超参数是否可以很好地拟合数据。  
这看起来是很简单，但要顺利地完成必须避过很多坑。

## 1. 什么是模型验证

### 1.1. 错误的模型验证方法

### 1.2. 模型验证正确方法：留出集

### 1.3. 交叉检验

## 2. 选择最优模型

### 2.1. 偏差与方差的均衡

### 2.2. Scikit-Learn验证曲线

## 3. 学习曲线

### 3.1. Scikit-Learn学习曲线

## 4. 验证实践：网格搜索